In [39]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import time

import requests
from bs4 import BeautifulSoup

In [69]:
def get_basic_position(style):

    splitted = style.split('(')
    width = float(splitted[2].split('%')[0])
    height = float(splitted[1].split('%')[0])
    
    group = ""
    
    if width < 10:
        group = "GK"
    elif width < 20:
        group = "DF"
    elif width < 40:
        group = "MF"
    else:
        group = "FW"
  
    return group

In [85]:
def get_detailed_position(style, gk, df, mf, fw):
    
    splitted = style.split('(')
    width = float(splitted[2].split('%')[0])
    height = float(splitted[1].split('%')[0])
    
    right_position = ""
    
    if width < 10:
        position = "GK"
    elif width < 20:
        if df > 3:
            if height < 20 or height > 80:
                position = "WB"
            else:
                position = "CB"
        else:
            position = "CB"
    elif width == 25:
        if height < 20 or height > 80:
            position = "WB"
        else:
            position = "DM"
    elif width == 30:
        if mf == 3:
            position = "CM"
        else:
            if height < 20 or height > 80:
                position = "WM"
            else:
                position = "CM"
    elif width == 35:
        if height == 50:
            position = "AM"
        else:
            position = "WF"
    else:
        if height < 20 or height > 80:
            position = "WF"
        else:
            position = "ST"
    
    return position

In [97]:
def get_home_lineup(comp, year, home_team, away_team):
    
    from scores_and_fixtures import season_results_and_fixtures
    from datetime import datetime
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup

    results_df = season_results_and_fixtures(comp, year)
    
    for i, row in results_df.iterrows():
        if row['home_team'].lower() == home_team.lower() and row['away_team'].lower() == away_team.lower():
            url = f"https://fbref.com/{row['match_url']}"
            resp = requests.get(url) 
            resp.encoding = 'utf-8'
            soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')
            
            div_lineup_home = soup.find('div', {'class':'lineup'}, {'id':'a'})
            numbers, player_names, positions = [], [], []
            
            players = div_lineup_home.find_all('td')
                
            for i, value in enumerate(players):
                if i % 2 == 0:
                    numbers.append(value.get_text())
                elif i % 2 == 1: 
                    player_names.append(value.get_text())
                    
            df = pd.DataFrame(list(zip(numbers, player_names)), columns = ['squad_number', 'player'])
            
            positions = []
            
            for i, row in df.iterrows():
                try:
                    player_div = soup.find("div", {"title":{row['player']}})
                    style = player_div["style"]
                    get_position = get_basic_position(style)
                    df.at[i, 'position'] = get_position
                    positions.append(get_position)
                except: pass
                
            keepers = positions.count("GK")
            defenders = positions.count("DF")
            midfielders = positions.count("MF")
            forwards = positions.count("FW")
                
            for i, row in df.iterrows():
                try:
                    player_div = soup.find("div", {"title":{row['player']}})
                    style = player_div["style"]
                    get_det_position = get_detailed_position(style, keepers, defenders, midfielders, forwards)
                    df.at[i, 'det_pos'] = get_det_position
                except: pass

    return df

In [98]:
get_home_lineup("eng", 2023, "Arsenal", "Liverpool")

/Users/ivanbarin-turica/FBREF-Scraper/scores_and_fixtures.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


,squad_number,player,position,det_pos
0,22,David Raya,GK,GK
1,2,William Saliba,DF,CB
2,4,Ben White,DF,WB
3,6,Gabriel Magalhães,DF,CB
4,7,Bukayo Saka,FW,WF
5,8,Martin Ødegaard,MF,CM
6,11,Gabriel Martinelli,FW,WF
7,20,Jorginho,MF,CM
8,29,Kai Havertz,FW,ST
9,35,Oleksandr Zinchenko,DF,WB
